# Basic Libraries

In [3]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

import math
import random

from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer.noise import thermal_relaxation_error, ReadoutError, depolarizing_error

# provider = IBMQ.get_provider(hub='ibm-q')
# machine = provider.get_backend('ibmq_valencia')
from qiskit.test.mock import FakeBogota
machine = FakeBogota()
machine_noise_model = NoiseModel.from_backend(machine)
simulator = Aer.get_backend('qasm_simulator')


basis_gates = machine_noise_model.basis_gates

ibmqfactory.load_account:WARNING:2020-12-06 13:35:02,946: Credentials are already in use. The existing account in the session will be replaced.


# Important Functions

In [11]:
#Local Hamiltonian Function
from VQEHamiltonian_funcs import Hammifier

# Expectation Value/Running Functions
from Result_funcs import binaryToDecimal, dictToList, sampleExpecVal, countToEig, expecValForSamples, addDicts, evaluator

# Noise Model

In [12]:
'''

def generateDepolarizingError(machine, gate, qubits)

def generateRelaxationError(machine, gate, qubits, t1, t2, amp = 1, custom_t = False):
    """
    Return a relaxation error
    """

def generateNoiseModel(machine, coherent = True, incoherent = False, readout = False, custom_t = False, t1 = None, t2 = None, reverse = False):
    """
    Returns a realistic copy of london noise model with custom t1, t2 times
    """

'''

from NoiseModel import generateDepolarizingError, generateRelaxationError, generateNoiseModel

In [13]:
 noise_model = generateNoiseModel(machine, coherent = False, incoherent = False, readout= False)

# Restricted Range Info

In [14]:
'''------------Interatomic Distance = 0.74--------------
HF energy: -1.831863646477506
# of electrons: 2
# of spin orbitals: 4
Number of qubits are : 2
[ 0.00172129 -0.00182397 -0.11217598]
VQE Result: -1.1011690791894007 Exact Energy: -1.1372838344885012
All energies have been calculated'''

'------------Interatomic Distance = 0.74--------------\nHF energy: -1.831863646477506\n# of electrons: 2\n# of spin orbitals: 4\nNumber of qubits are : 2\n[ 0.00172129 -0.00182397 -0.11217598]\nVQE Result: -1.1011690791894007 Exact Energy: -1.1372838344885012\nAll energies have been calculated'

# Variational Form
For Hydrogen

In [15]:
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

initial_state = HartreeFock(
4,
2,
'parity'
) 
var_form = UCCSD(
            num_orbitals=4,
            num_particles=2,
            initial_state=initial_state,
            qubit_mapping='parity'
        )

# Running Circuits for different Param

In [16]:
counter = 0

def EnergyFinder(param):
    '''
    Input: param (1D array) for which to compute energy on VQE circ of H2
    Output: energy
    '''
    
    Hamiltonian = """II\nIZ\nZI\nZZ\nXX"""
    Hamiltonian_weights = [-1.053, 0.395, -0.395, -0.011, 0.181]
    backend = simulator
    global noise_model
    samples = 8192
    
    
    
    circ = var_form.construct_circuit(param).decompose()
    circuits = Hammifier(circ, Hamiltonian)
    noisy = evaluator(circuits, samples, noise_model, Hamiltonian, Hamiltonian_weights, simulator)
    ideal = evaluator(circuits, samples, None, Hamiltonian, Hamiltonian_weights, simulator)
    return noisy, ideal

def main():
    '''
    Choose param and compute mean
    '''
    d = -math.pi
    u = math.pi
    
    energies = []
    diff = []
    params = []
    
    for i in range(1000):
        print(i)
    
        p1 = random.uniform(0.001, 0.002)
        p2 = random.uniform(-0.002, -0.001)
        p3 = random.uniform(-0.2, -0.1)


        noisy, ideal = EnergyFinder([p1,p2,p3])
        energies.append([[p1,p2,p3], ideal, noisy])
        diff.append(noisy-ideal)
        
    return energies, diff
energies, diff = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
import numpy as np
import matplotlib.pyplot as plt

a = plt.figure()
plt.hist(diff, bins=100, color = "darkgoldenrod")
plt.title("Noisy - Ideal Energy (Restricted Range w/n R,C,I)")
plt.xlabel("Energy Difference (Hartree)")
plt.ylabel("Frequency")
plt.show()
a.savefig('Restricted_Range_without_RCI.png', dpi = 1000)

In [18]:
energies

[[[0.0012394474785049046, -0.0011586498426958876, -0.13689605128240856],
  -1.0868758855282934,
  -1.0859759831845435],
 [[0.0011447479835423786, -0.0012054841239006494, -0.12239850459404467],
  -1.0946556706845434,
  -1.0880943913876684],
 [[0.001653046228804318, -0.0012179836647232037, -0.11865739456842078],
  -1.0929317937314185,
  -1.0956835027157934],
 [[0.0011283561051124375, -0.0010408204897081079, -0.19009726676687233],
  -1.0591793523251685,
  -1.0610399480282933],
 [[0.0010143966719328608, -0.0013929502007202644, -0.18542764534674108],
  -1.0562264714657934,
  -1.0589173894345434],
 [[0.001462812080467935, -0.0011116136837857104, -0.1900104905091627],
  -1.0634059148251684,
  -1.0546029363095435],
 [[0.0019181585267294963, -0.0015221249913438895, -0.1164583949181373],
  -1.0938478093564183,
  -1.0931527409970436],
 [[0.0014312934652072919, -0.001794883659081844, -0.12905714212889347],
  -1.0881068425595435,
  -1.0957801824032933],
 [[0.0018161603168981937, -0.0015888981383922

In [19]:
diff

[0.0008999023437499254,
 0.006561279296875,
 -0.0027517089843749343,
 -0.001860595703124801,
 -0.002690917968749984,
 0.008802978515624948,
 0.0006950683593747176,
 -0.007673339843749849,
 -0.006456298828124707,
 0.0047485351562497335,
 4.0039062500163425e-05,
 -0.0042575683593748526,
 -0.005789794921875258,
 -0.0009477539062499041,
 -0.0022739257812500746,
 0.00957861328124987,
 4.003906249994138e-05,
 -0.0102543945312501,
 -0.003050048828125096,
 0.0013146972656250533,
 0.01034228515625002,
 0.0025639648437498774,
 -0.0007226562500002309,
 0.0031430664062499503,
 -0.009779785156249998,
 -0.005858398437500023,
 0.003380371093750334,
 -0.0025468749999999485,
 -0.005134277343749938,
 0.005636474609375108,
 0.006678222656250066,
 0.00204833984375008,
 -0.007393554687500137,
 -0.001788818359375055,
 0.006058593750000174,
 0.007033935546874925,
 -0.007390136718750107,
 0.003029296875000087,
 0.004399169921874835,
 -0.0022094726562500444,
 -0.0021777343750000178,
 0.0014321289062499964,
 -0

# Without Readout